# 01_data_cleaning.ipynb

UberEats Order Data Cleaning & Curation  
Authors: Sarvesha  |  Project X

This notebook expects the following folder layout on your local machine:
```
project_x/
├── data_raw/
│   └── user_orders-0.csv
├── data_cleaned/
│   └── (output will appear here)
├── notebooks/
│   └── 01_data_cleaning.ipynb
├── requirements.txt
└── README.md
```
Run this notebook from the `notebooks/` folder. It will automatically find the project root, create missing folders, and write outputs without manual edits.

In [98]:
# ─────────────────────────────────────────────────────────────
# Path setup (portable: no manual edits required)
# ─────────────────────────────────────────────────────────────
from pathlib import Path
import sys

cwd = Path.cwd()

# If running from project_x/notebooks, root is parent; otherwise try to detect
if cwd.name == 'notebooks' and (cwd.parent / 'data_raw').exists():
    ROOT = cwd.parent
else:
    # Fallback: search upwards for a directory containing data_raw
    search = cwd
    ROOT = None
    for _ in range(5):  # search up to 5 levels up
        if (search / 'data_raw').exists():
            ROOT = search
            break
        search = search.parent
    if ROOT is None:
        # If not found, assume current working dir is root
        ROOT = cwd

DATA_RAW = ROOT / 'data_raw'
DATA_CLEANED = ROOT / 'data_cleaned'
DATA_CLEANED.mkdir(parents=True, exist_ok=True)

INPUT_CSV = DATA_RAW / 'user_orders-0.csv'
OUTPUT_CSV = DATA_CLEANED / 'user_orders_clean.csv'
REPORT_PATH = DATA_CLEANED / 'cleaning_report.txt'

print('Project root :', ROOT)
print('Input CSV    :', INPUT_CSV)
print('Output CSV   :', OUTPUT_CSV)
assert INPUT_CSV.exists(), f"Expected input file at: {INPUT_CSV}"


Project root : /Users/sarveshaks/Documents/Documents - Sarvesha’s MacBook Pro (2)/Project Stea/project_x
Input CSV    : /Users/sarveshaks/Documents/Documents - Sarvesha’s MacBook Pro (2)/Project Stea/project_x/data_raw/user_orders-0.csv
Output CSV   : /Users/sarveshaks/Documents/Documents - Sarvesha’s MacBook Pro (2)/Project Stea/project_x/data_cleaned/user_orders_clean.csv


In [99]:
# ─────────────────────────────────────────────────────────────
# Imports
# ─────────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import re
from langdetect import detect
from tqdm import tqdm

tqdm.pandas()


In [100]:
# ─────────────────────────────────────────────────────────────
# Load
# ─────────────────────────────────────────────────────────────
def read_csv_robust(path):
    try:
        print("im in")
        return pd.read_csv(INPUT_CSV, engine='python', on_bad_lines='skip')
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding='latin-1')

df = read_csv_robust(INPUT_CSV) 
orig_shape = df.shape
display(df.head())
print('Initial shape:', orig_shape)


im in


,City_Name,Restaurant_Name,Request_Time_Local,Final_Delivery_Time_Local,Order_Status,Item_Name,Item_quantity,Customizations,Customization_Cost_Local,Special_Instructions,Item_Price,Order_Price,Currency
0,Portland,Carl's Jr. (8205 SE Rhine St.),2025-03-01T01:04:56.000Z,2025-03-01T01:45:16.000Z,completed,Big Angus El Diablo Combo,1.0,"CHOOSE A DRINK, CHOOSE A SIDE, Coca-Cola®, Nat...",0.00,NaN,18.24,22.24,USD
1,Portland,Carl's Jr. (8205 SE Rhine St.),2025-03-01T01:04:56.000Z,2025-03-01T01:45:16.000Z,completed,Big Angus El Diablo Combo,1.0,Medium,1.25,NaN,18.24,22.24,USD
2,Portland,Flavours Of India,2025-01-29T20:11:02.000Z,2025-01-29T21:00:35.000Z,completed,Dal Makhani,1.0,NaN,NaN,NaN,16.99,34.53,USD
3,Portland,Flavours Of India,2025-01-29T20:11:02.000Z,2025-01-29T21:00:35.000Z,completed,Paneer Tikka Kabob,1.0,NaN,NaN,NaN,17.99,34.53,USD
4,Portland,Chennai Masala (Hillsboro),2025-01-05T19:31:16.000Z,2025-01-05T20:28:16.000Z,completed,Idly Sambar,1.0,NaN,NaN,NaN,8.50,31.70,USD


Initial shape: (2463, 13)


In [101]:
df.columns

Index(['City_Name', 'Restaurant_Name', 'Request_Time_Local',
       'Final_Delivery_Time_Local', 'Order_Status', 'Item_Name',
       'Item_quantity', 'Customizations', 'Customization_Cost_Local',
       'Special_Instructions', 'Item_Price', 'Order_Price', 'Currency'],
      dtype='object')

In [102]:
# ─────────────────────────────────────────────────────────────
# Column normalization (specific to Uber Eats schema)
# ─────────────────────────────────────────────────────────────
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# Explicitly use Item_Name as the dish column
if "item_name" not in df.columns:
    raise KeyError("Expected column 'Item_Name' not found in CSV")

df.rename(columns={"item_name": "dish_name"}, inplace=True)

# Drop duplicates and NaN values in dish_name
before_dropna = len(df)
df.dropna(subset=["dish_name"], inplace=True)
df["dish_name"] = df["dish_name"].astype(str).str.strip()
df.drop_duplicates(subset=["dish_name"], inplace=True)

print(f"Dropped {before_dropna - len(df)} rows due to NaNs/duplicates in dish_name")

# Show a quick sample for sanity check
display(df[["city_name", "restaurant_name", "dish_name"]].head())



Dropped 1541 rows due to NaNs/duplicates in dish_name


,city_name,restaurant_name,dish_name
0,Portland,Carl's Jr. (8205 SE Rhine St.),Big Angus El Diablo Combo
2,Portland,Flavours Of India,Dal Makhani
3,Portland,Flavours Of India,Paneer Tikka Kabob
4,Portland,Chennai Masala (Hillsboro),Idly Sambar
5,Portland,Chennai Masala (Hillsboro),Mysore Masala Dosa


In [103]:
df.shape

(922, 13)

In [104]:
# ─────────────────────────────────────────────────────────────
# Language filter (English only)
# ─────────────────────────────────────────────────────────────
def is_english(text: str) -> bool:
    try:
        return detect(text) == 'en'
    except Exception:
        return False

df['is_english'] = df['dish_name'].progress_apply(is_english)
df = df[df['is_english']].drop(columns=['is_english'])
print('After language filter:', df.shape)


100%|██████████| 922/922 [00:01<00:00, 503.84it/s]

After language filter: (384, 13)


In [105]:
df.head()

,city_name,restaurant_name,request_time_local,final_delivery_time_local,order_status,dish_name,item_quantity,customizations,customization_cost_local,special_instructions,item_price,order_price,currency
6,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Chicken shawarma gyro,1.0,Spicy Sauce,0.00,NaN,13.95,70.80,USD
8,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Crispy Fries,1.0,NaN,NaN,NaN,9.95,70.80,USD
9,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Chicken Shawarma Plate,1.0,NaN,NaN,NaN,17.95,70.80,USD
10,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Falafel Over Rice Plate,2.0,NaN,NaN,NaN,35.90,70.80,USD
11,Portland,Jack in the Box (4242 Se 82Nd Ave),2024-11-28T01:09:36.000Z,2024-11-28T01:18:20.000Z,completed,Jack's Spicy Chicken® w/ Cheese,1.0,Extra Swiss Cheese,0.75,NaN,10.61,25.05,USD


In [106]:
# ─────────────────────────────────────────────────────────────
# Cleaning Pipeline: Regex, Functions, and Application
# ─────────────────────────────────────────────────────────────
import regex
import unicodedata
from typing import Set, Final, Pattern

# --- 1. Helper Functions ---

def strip_accents(text: str) -> str:
    """Strips accents from a string and converts to ASCII."""
    try:
        nfkd_form = unicodedata.normalize("NFKD", str(text))
        return "".join(
            [c for c in nfkd_form if not unicodedata.combining(c)]
        )
    except TypeError:
        return ""

# --- 2. Compiled Regex Patterns ---

# Remove parentheticals
RE_PAREN: Final[Pattern] = regex.compile(r"[\(\[].*?[\)\]]")

# Normalize symbols
NORMALIZE_MAP: Final[dict[str, str]] = {
    "w/": " with ",
    "&": " and ",
}

# Remove quantities, units, and serving sizes
RE_QUANTITY: Final[Pattern] = regex.compile(
    r"(\b(serves?|feeds?)\s*\d+\b"  # serves 2, feeds 4
    r"|\b\d+\s*for\s*\$?\d+(\.\d+)?\b"  # 2 for $5
    r"|\b(x\d+|\d+x)\b"  # x2, 2x
    # 20pc, 12 ct, 8oz, 500g, 1.5 oz (with or without space)
    r"|\b\d+(\.\d+)?\s*(pc|pcs|ct|oz|lb|lbs|kg|g|ml|l|inch|in|'|\")s?\b"
    r"|\b\d+(\.\d+)?s?\b)",  # standalone numbers
    flags=regex.IGNORECASE,
)

# Remove instruction phrases
RE_INSTRUCTION: Final[Pattern] = regex.compile(
    r"(\b(no|without|w/o|extra|add|added|light|less|more|sub|swap)\s+[\p{L}]+\b"
    r"|\bon\s+the\s+side\b)",  # 'on the side'
    flags=regex.IGNORECASE,
)

# Marketing, packaging, and other noise tokens
MARKETING_BLOCKLIST: Final[Set[str]] = {
    "appetizer", "add", "added", "box", "bucket", "combo", "ct",
    "deal", "deluxe", "dessert", "double", "entree", "family",
    "feeds", "for", "g", "in", "inch", "jumbo", "junior", "kids",
    "kg", "l", "large", "lb", "lbs", "meal", "medium", "mini",
    "ml", "oz", "pack", "party", "pc", "pcs", "platter", "regular",
    "serves", "single", "small", "special", "starter", "tray",
    "triple", "value", "xl", "xxl", "side"
}
RE_MARKETING: Final[Pattern] = regex.compile(
    r"\b(" + r"|".join(MARKETING_BLOCKLIST) + r")\b", flags=regex.IGNORECASE
)

# Final cleanup: remove all non-letter, non-space characters
RE_FINAL_SYMBOLS: Final[Pattern] = regex.compile(r"[^a-z\s]")

# Domain-specific 1-char tokens to keep
DOMAIN_1CHAR_EXCEPTIONS: Final[Set[str]] = {"bbq"}

# --- 3. Cleaning Orchestration Functions ---

def build_restaurant_blocklist(name: str) -> Set[str]:
    """Creates a set of noise tokens from a restaurant name."""
    if not name or not isinstance(name, str):
        return set()
    clean_name = strip_accents(name.lower())
    clean_name = RE_FINAL_SYMBOLS.sub(" ", clean_name)
    tokens = clean_name.split()
    return {token for token in tokens if len(token) >= 3}

def _final_token_cleanup(text: str) -> str:
    """De-duplicates tokens and removes 1-char tokens."""
    if not text:
        return ""
    tokens = text.split()
    final_tokens = []
    last_token = None
    for token in tokens:
        if token == last_token:
            continue
        if len(token) > 1 or token in DOMAIN_1CHAR_EXCEPTIONS:
            final_tokens.append(token)
        last_token = token
    return " ".join(final_tokens)

def clean_dish_name(name: str, restaurant_tokens: Set[str]) -> str:
    """Main cleaning orchestrator function."""
    if not name or not isinstance(name, str):
        return ""

    # 1. ASCII conversion and lowercasing
    clean_name = strip_accents(name.lower())
    # 2. Remove parentheticals
    clean_name = RE_PAREN.sub(" ", clean_name)
    # 3. Normalize common symbols
    for sym, replacement in NORMALIZE_MAP.items():
        clean_name = clean_name.replace(sym, replacement)
    # 4. Remove quantities and units
    clean_name = RE_QUANTITY.sub(" ", clean_name)
    # 5. Remove instruction phrases
    clean_name = RE_INSTRUCTION.sub(" ", clean_name)
    # 6. Remove marketing/packaging words
    clean_name = RE_MARKETING.sub(" ", clean_name)
    # 7. Remove restaurant-specific tokens (if any)
    if restaurant_tokens:
        try:
            # Build regex on-the-fly for this row's restaurant
            restaurant_re = regex.compile(
                r"\b(" + r"|".join(regex.escape(token) for token in restaurant_tokens) + r")\b",
                flags=regex.IGNORECASE
            )
            clean_name = restaurant_re.sub(" ", clean_name)
        except (regex.error, TypeError):
            pass # Ignore if regex fails
    # 8. Final symbol strip and whitespace collapse
    clean_name = RE_FINAL_SYMBOLS.sub(" ", clean_name)
    clean_name = regex.sub(r"\s+", " ", clean_name).strip()
    # 9. Final token-level cleanup (dedup, 1-char removal)
    clean_name = _final_token_cleanup(clean_name)

    return clean_name

# --- 4. Apply Pipeline to DataFrame ---

print("Building restaurant token blocklists...")
# Fill NaNs in restaurant_name just in case
df['restaurant_name'] = df['restaurant_name'].fillna("")
df['restaurant_tokens'] = df['restaurant_name'].progress_apply(build_restaurant_blocklist)

print("Applying main cleaning pipeline...")
df['dish_name_clean'] = df.progress_apply(
    lambda row: clean_dish_name(row['dish_name'], row['restaurant_tokens']),
    axis=1
)

# --- 5. Final Quality Gate ---
print(f"Shape before empty-name drop: {df.shape}")
df_clean = df[
    (df['dish_name_clean'].notna()) & (df['dish_name_clean'] != "")
].copy()
print(f"Shape after empty-name drop: {df_clean.shape}")

# --- 6. Show Results ---
print("\n--- Cleaning Examples (Before -> After) ---")
display(df_clean[['dish_name', 'dish_name_clean', 'restaurant_name']].sample(n=min(20, len(df_clean)), random_state=42))

Building restaurant token blocklists...


100%|██████████| 384/384 [00:00<00:00, 243663.05it/s]


Applying main cleaning pipeline...


100%|██████████| 384/384 [00:00<00:00, 50540.13it/s]

Shape before empty-name drop: (384, 15)
Shape after empty-name drop: (382, 15)

--- Cleaning Examples (Before -> After) ---


,dish_name,dish_name_clean,restaurant_name
1832,Lamb over Rice,lamb over rice,Naz's Halal Food - Burtonsville
1555,Spicy Ghost Chicken Curry House Special,spicy ghost chicken house,Curry Place Silver Spring
1193,Chipotle Cheddar Chalupa - Black Bean,chipotle cheddar chalupa black bean,Taco Bell (11017 Nokesville Road)
2254,Create Your Own Mine,create your own mine,Cold Stone (8113 Sudley Rd)
233,Sri Lankan Iced Coffee,sri lankan iced coffee,Mirisata
2443,3 Hotcakes,hotcakes,McDonald's® (College Park)
155,Chocolate Chip,chocolate chip,Subway (4328 SE 82nd Ave)
498,Large FUZE® Iced Tea,fuze iced tea,Jack in the Box (4242 Se 82Nd Ave)
2450,Chicken Alfredo,chicken alfredo,Pizza Mart (College Park)
1461,Ketchup Packet,ketchup packet,McDonald's® (9001 CENTERVILLE RD)[RENOVATION H...


In [107]:
# ─────────────────────────────────────────────────────────────
# Cleaning "customizations" and creating "dish_name_to_be_processed"
# ─────────────────────────────────────────────────────────────
import pandas as pd
import regex
from typing import Set

# --- 1. Define New Regex for Customization-Specific Noise ---

# Customizations often have all-caps instructional text.
# This regex targets phrases with 2+ all-caps words.
RE_CAPS_INSTRUCTION: Final[Pattern] = regex.compile(
    r"\b([A-Z]{2,}\s+){1,}[A-Z]{2,}\b"  # e.g., "CHOOSE A DRINK"
)

# --- 2. Define Customization Cleaning Function ---

def clean_customizations(cust_str: str, restaurant_tokens: Set[str]) -> str:
    """
    Applies the full cleaning pipeline to the 'customizations' string.
    
    This re-uses the regex patterns defined in the previous cell 
    (RE_PAREN, NORMALIZE_MAP, RE_QUANTITY, etc.)
    """
    if not cust_str or not isinstance(cust_str, str):
        return ""

    # 1. Start with ASCII conversion and lowercasing
    clean_name = strip_accents(cust_str.lower())
    
    # 2. Remove new all-caps instructional noise
    clean_name = RE_CAPS_INSTRUCTION.sub(" ", clean_name)
    
    # 3. Remove parentheticals
    clean_name = RE_PAREN.sub(" ", clean_name)
    
    # 4. Normalize common symbols
    for sym, replacement in NORMALIZE_MAP.items():
        clean_name = clean_name.replace(sym, replacement)
        
    # 5. Remove quantities and units
    clean_name = RE_QUANTITY.sub(" ", clean_name)
    
    # 6. Remove instruction phrases
    clean_name = RE_INSTRUCTION.sub(" ", clean_name)
    
    # 7. Remove marketing/packaging words
    clean_name = RE_MARKETING.sub(" ", clean_name)
    
    # 8. Remove restaurant-specific tokens (if any)
    if restaurant_tokens:
        try:
            restaurant_re = regex.compile(
                r"\b(" + r"|".join(regex.escape(token) for token in restaurant_tokens) + r")\b",
                flags=regex.IGNORECASE
            )
            clean_name = restaurant_re.sub(" ", clean_name)
        except (regex.error, TypeError):
            pass
            
    # 9. Final symbol strip and whitespace collapse
    clean_name = RE_FINAL_SYMBOLS.sub(" ", clean_name)
    # Replace commas (common in customizations) with spaces *before* collapsing
    clean_name = clean_name.replace(",", " ")
    clean_name = regex.sub(r"\s+", " ", clean_name).strip()
    
    # 10. Final token-level cleanup (dedup, 1-char removal)
    clean_name = _final_token_cleanup(clean_name)

    return clean_name

# --- 3. Apply Customization Cleaning ---

# Use df_clean from the previous step.
# We need to make sure we're not modifying a copy.
df_processed = df_clean.copy()

print("Cleaning 'customizations' column...")
# Fill NaN values in 'customizations' to avoid errors
df_processed['customizations'] = df_processed['customizations'].fillna("")

df_processed['customization_clean'] = df_processed.progress_apply(
    lambda row: clean_customizations(row['customizations'], row['restaurant_tokens']),
    axis=1
)

# --- 4. Combine Columns into "dish_name_to_be_processed" ---

def create_processed_name(row: pd.Series) -> str:
    """
    Combines dish_name_clean and customization_clean based on the rules.
    
    - If customization_clean is empty (was only noise), return dish_name_clean.
    - If customization_clean has content, return "dish_name_clean customization_clean".
    """
    dish_name = row['dish_name_clean']
    cust_clean = row['customization_clean']
    
    if cust_clean:  # True if the string is not empty
        return f"{dish_name} {cust_clean}"
    else:
        return dish_name

print("Creating 'dish_name_to_be_processed' column...")
df_processed['dish_name_to_be_processed'] = df_processed.apply(create_processed_name, axis=1)

# --- 5. Show Results ---
print("\n--- Final Combination Examples ---")
display(df_processed[
    ['dish_name_clean', 'customizations', 'customization_clean', 'dish_name_to_be_processed']
].sample(n=min(20, len(df_processed)), random_state=42))

# Display a specific example of "noise-only" customization
noise_example = df_processed[df_processed['customizations'].str.contains("CHOOSE A DRINK", na=False)]
print("\n--- Example of 'Noise-Only' Customization ---")
display(noise_example[
    ['dish_name_clean', 'customizations', 'customization_clean', 'dish_name_to_be_processed']
].head())

# Assign back to df_clean if you want to continue using that variable
df_clean = df_processed.copy()

Cleaning 'customizations' column...


100%|██████████| 382/382 [00:00<00:00, 65221.21it/s]

Creating 'dish_name_to_be_processed' column...

--- Final Combination Examples ---


,dish_name_clean,customizations,customization_clean,dish_name_to_be_processed
1832,lamb over rice,"Green Sauce, White Sauce",green sauce white sauce,lamb over rice green sauce white sauce
1555,spicy ghost chicken house,,,spicy ghost chicken house
1193,chipotle cheddar chalupa black bean,"No Lettuce, No Tomatoes",,chipotle cheddar chalupa black bean
2254,create your own mine,"Chocolate Chips, M&M's, Kit Kat, Pistachio Ice...",chocolate chips and kit kat pistachio ice cera...,create your own mine chocolate chips and kit k...
233,sri lankan iced coffee,,,sri lankan iced coffee
2443,hotcakes,,,hotcakes
155,chocolate chip,,,chocolate chip
498,fuze iced tea,Raspberry Peach,raspberry peach,fuze iced tea raspberry peach
2450,chicken alfredo,,,chicken alfredo
1461,ketchup packet,,,ketchup packet



--- Example of 'Noise-Only' Customization ---


,dish_name_clean,customizations,customization_clean,dish_name_to_be_processed


In [108]:
df = df_clean.copy()

df.head()

,city_name,restaurant_name,request_time_local,final_delivery_time_local,order_status,dish_name,item_quantity,customizations,customization_cost_local,special_instructions,item_price,order_price,currency,restaurant_tokens,dish_name_clean,customization_clean,dish_name_to_be_processed
6,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Chicken shawarma gyro,1.0,Spicy Sauce,0.00,NaN,13.95,70.80,USD,"{davis, kingdom, gyro}",chicken shawarma,spicy sauce,chicken shawarma spicy sauce
8,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Crispy Fries,1.0,,NaN,NaN,9.95,70.80,USD,"{davis, kingdom, gyro}",crispy fries,,crispy fries
9,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Chicken Shawarma Plate,1.0,,NaN,NaN,17.95,70.80,USD,"{davis, kingdom, gyro}",chicken shawarma plate,,chicken shawarma plate
10,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Falafel Over Rice Plate,2.0,,NaN,NaN,35.90,70.80,USD,"{davis, kingdom, gyro}",falafel over rice plate,,falafel over rice plate
11,Portland,Jack in the Box (4242 Se 82Nd Ave),2024-11-28T01:09:36.000Z,2024-11-28T01:18:20.000Z,completed,Jack's Spicy Chicken® w/ Cheese,1.0,Extra Swiss Cheese,0.75,NaN,10.61,25.05,USD,"{box, the, ave, jack}",spicy chicken with cheese,cheese,spicy chicken with cheese cheese


In [109]:
# ─────────────────────────────────────────────────────────────
# Optional placeholders (for Week-2 augmentation)
# ─────────────────────────────────────────────────────────────
df['dish_description'] = np.nan
df['sugar_level'] = np.nan
df['sodium_level'] = np.nan

df.sample(5)


,city_name,restaurant_name,request_time_local,final_delivery_time_local,order_status,dish_name,item_quantity,customizations,customization_cost_local,special_instructions,item_price,order_price,currency,restaurant_tokens,dish_name_clean,customization_clean,dish_name_to_be_processed,dish_description,sugar_level,sodium_level
1411,Washington D.C.,Saravana Palace,2022-08-15T12:53:22.000Z,2022-08-15T13:51:33.000Z,completed,Puri Specials with Channa,1.0,,NaN,NaN,12.99,131.41,USD,"{palace, saravana}",puri specials with channa,,puri specials with channa,NaN,NaN,NaN
1833,Washington D.C.,McDonald's® (College Park),2021-01-19T23:34:54.000Z,2021-01-19T23:47:44.000Z,completed,Bottled Orange Juice,1.0,,NaN,NaN,2.69,18.60,USD,"{college, park, mcdonald}",bottled orange juice,,bottled orange juice,NaN,NaN,NaN
125,Portland,Jack in the Box (4242 Se 82Nd Ave),2024-01-20T20:10:47.000Z,2024-01-20T20:44:57.000Z,completed,Large Chicken Fajita Pita Combo,1.0,"Large Fanta®, Extra Salsa, Large French Fry, N...",0.0,NaN,15.48,39.51,USD,"{box, the, ave, jack}",chicken fajita pita,fanta french fry easy tomato,chicken fajita pita fanta french fry easy tomato,NaN,NaN,NaN
398,Portland,Shake Shack (West End Portland),2023-10-24T20:13:31.000Z,2023-10-24T20:51:19.000Z,completed,Kiwi Apple Limeade,1.0,Large,1.5,NaN,3.69,32.60,USD,"{west, end, shake, portland, shack}",kiwi apple limeade,,kiwi apple limeade,NaN,NaN,NaN
198,Portland,Sunny & Fine's Breakfast Burritos (11475 SW Pa...,2024-01-05T11:29:06.000Z,2024-01-05T12:15:32.000Z,completed,Hash Browns,1.0,,NaN,NaN,3.99,48.56,USD,"{hwy, sunny, pacific, burritos, fine, breakfast}",hash browns,,hash browns,NaN,NaN,NaN


In [110]:
df.head()

,city_name,restaurant_name,request_time_local,final_delivery_time_local,order_status,dish_name,item_quantity,customizations,customization_cost_local,special_instructions,item_price,order_price,currency,restaurant_tokens,dish_name_clean,customization_clean,dish_name_to_be_processed,dish_description,sugar_level,sodium_level
6,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Chicken shawarma gyro,1.0,Spicy Sauce,0.00,NaN,13.95,70.80,USD,"{davis, kingdom, gyro}",chicken shawarma,spicy sauce,chicken shawarma spicy sauce,NaN,NaN,NaN
8,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Crispy Fries,1.0,,NaN,NaN,9.95,70.80,USD,"{davis, kingdom, gyro}",crispy fries,,crispy fries,NaN,NaN,NaN
9,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Chicken Shawarma Plate,1.0,,NaN,NaN,17.95,70.80,USD,"{davis, kingdom, gyro}",chicken shawarma plate,,chicken shawarma plate,NaN,NaN,NaN
10,Portland,Gyro Kingdom (NE Davis),2024-12-01T02:37:08.000Z,2024-12-01T02:56:34.000Z,completed,Falafel Over Rice Plate,2.0,,NaN,NaN,35.90,70.80,USD,"{davis, kingdom, gyro}",falafel over rice plate,,falafel over rice plate,NaN,NaN,NaN
11,Portland,Jack in the Box (4242 Se 82Nd Ave),2024-11-28T01:09:36.000Z,2024-11-28T01:18:20.000Z,completed,Jack's Spicy Chicken® w/ Cheese,1.0,Extra Swiss Cheese,0.75,NaN,10.61,25.05,USD,"{box, the, ave, jack}",spicy chicken with cheese,cheese,spicy chicken with cheese cheese,NaN,NaN,NaN


In [111]:
# ─────────────────────────────────────────────────────────────
# Save outputs + a short cleaning report
# ─────────────────────────────────────────────────────────────
df.to_csv(OUTPUT_CSV, index=False)
print('✅ Cleaned dataset saved to:', OUTPUT_CSV)

report = []
report.append(f"Project root        : {ROOT}")
report.append(f"Input CSV           : {INPUT_CSV}")
report.append(f"Output CSV          : {OUTPUT_CSV}")
report.append(f"Original shape      : {orig_shape}")
report.append(f"Final shape         : {df.shape}")
report.append("")
report.append("Column summary:")
for c in df.columns:
    report.append(f" - {c}: {df[c].notna().sum()} non-null")

REPORT_PATH.write_text("\n".join(report))
print('📝 Cleaning report  :', REPORT_PATH)


✅ Cleaned dataset saved to: /Users/sarveshaks/Documents/Documents - Sarvesha’s MacBook Pro (2)/Project Stea/project_x/data_cleaned/user_orders_clean.csv
📝 Cleaning report  : /Users/sarveshaks/Documents/Documents - Sarvesha’s MacBook Pro (2)/Project Stea/project_x/data_cleaned/cleaning_report.txt


In [112]:
df["dish_name_clean"]

6                      chicken shawarma
8                          crispy fries
9                chicken shawarma plate
10              falafel over rice plate
11            spicy chicken with cheese
                     ...               
2457           chicken cheesesteak roll
2458                     fried dumpling
2459        teriyaki chicken and shrimp
2461                    flat iron steak
2462    southern fried chicken sandwich
Name: dish_name_clean, Length: 382, dtype: object